# Customer & Client Experience Insights

This notebook explores customer and client service indicators to identify patterns that influence experience outcomes such as satisfaction and retention.
The objective is to translate operational service metrics into insights that support service improvement decisions.

## Business question

What patterns in customer and client behaviour help explain experience outcomes, and how can these insights inform service improvements?

## Data

This notebook uses `customer_experience_data.csv` located in the `data/` folder.


## Approach

- Explore how response and resolution times relate to satisfaction.
- Compare experience outcomes across customer segments.
- Flag experience-risk patterns for operational follow-up.
- Produce a single stakeholder-facing snapshot visual.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/customer_experience_data.csv")
df.head()

In [ ]:
# Basic distribution checks
df.describe(include='all').T.head(20)

In [ ]:
# Relationship between response time and satisfaction
df.groupby(pd.cut(df["response_time_hours"], [0,2,6,12,24,48,120]))["satisfaction_score"].mean()

In [ ]:
# Segment-level experience view
seg = df.groupby("segment", as_index=False).agg(
    customers=("customer_id","count"),
    avg_satisfaction=("satisfaction_score","mean"),
    avg_response=("response_time_hours","mean"),
    avg_resolution=("resolution_time_hours","mean"),
    retention_rate=("retained_next_period","mean")
)
seg

In [ ]:
# Identify experience risk: low satisfaction + long resolution
risk = df[(df["satisfaction_score"] <= 2.5) & (df["resolution_time_hours"] >= df["resolution_time_hours"].quantile(0.8))]
risk.sample(10, random_state=1)

In [ ]:
import matplotlib.pyplot as plt

# Visual: Satisfaction by response time band (stakeholder snapshot)
df["response_band"] = pd.cut(df["response_time_hours"], bins=[0,2,6,12,24,48,120],
                            labels=["<2h","2-6h","6-12h","12-24h","24-48h","48h+"])
band = df.groupby("response_band", observed=True)["satisfaction_score"].mean().reset_index()

plt.figure(figsize=(9,5))
plt.bar(band["response_band"].astype(str), band["satisfaction_score"])
plt.ylabel("Average satisfaction (1–5)")
plt.title("Average Satisfaction by First Response Time")
plt.tight_layout()

plt.savefig("../visuals/insight_snapshot.png", dpi=200)
plt.show()

In [ ]:
# Key findings summary table
band